# Python codes for [Introduction to Computional Stochastic PDEs, CUP 2014](http://www.cambridge.org/gb/academic/subjects/mathematics/differential-and-integral-equations-dynamical-systems-and-co/introduction-computational-stochastic-pdes?format=PB&isbn=9780521728522)

[T. Shardlow](http://people.bath.ac.uk/tjs42/) Mar 2016
# Chapter 4





In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
import sys
if sys.version_info < (3,):
    try:
        from builtins import (bytes, dict, int, list, object, range, str,
                              ascii, chr, hex, input, next, oct, open,
                              pow, round, super, filter, map, zip)
        from future.builtins.disabled import (apply, cmp, coerce, execfile,
                                              file, long, raw_input,
                                              reduce, reload,
                                              unicode, xrange, StandardError)
    except:
        print("need future module")


In [2]:
from math import *
# Numpy
import numpy as np
from numpy import matlib
# Plotting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
# Scipy
import scipy
from scipy import sparse
from scipy.sparse import linalg
from scipy import optimize

Chapter 4 starts the work with random numbers and the first function shows how to sample from the Gaussian distribution $N(\mu,\sigma)$ and compute the sample mean and variance.


In [3]:
def est_mean_var_func(mu,sigma,M):
    """
    A4.1 Page 163
    """
    X=np.random.randn(M)
    X=mu + sigma * X
    mu_M=np.mean(X)
    sig_sq_M=np.var(X)
    return mu_M,sig_sq_M

In [4]:
mu=1;    sigma=1; M=40
mu_M,sig_sq_M=est_mean_var_func(mu,sigma,M)
print("The sample average is", mu_M,
    "and the sample variance", sig_sq_M,
    "based on ", M, "samples for N(1,1)")

The sample average is 0.650211887244 and the sample variance 0.843164549377 based on  40 samples for N(1,1)


Python generates pseudo random numbers and the random number stream can be reproduced by storing and resetting the state of the random number generator.

In [5]:
def setseed0(M):
    """
    A4.2 Page 165
    """
    s0=np.random.RandomState()
    s0_state=s0.get_state()
    r0=s0.randn(M)
    s0.set_state(s0_state)
    r00=s0.randn(M)
    return r0,r00
    
def setseed1(M):
    """
    A4.3 Page 165
    """
    s1=np.random.RandomState()
    s0=np.random.RandomState()
    r1=s1.randn(M)
    r0=s0.randn(M)
    tmp=np.vstack((r1,
                   r0))
    return  np.cov(tmp)

We show how to reproduce a stream of random numers with `setseet0` and how to work with two uncorrelated random number streams in `setseed1`. Notice that the off-diagonals of the covariance are very small, indicating the samples are uncorrelated.

In [6]:
r0,r00=setseed0(10)
print("Identical samples:\n",r0,"and\n",r00)
#
print("Covariance of samples", setseed1(1000))

Identical samples:
 [ 0.81489992  0.76143959  0.07694935 -0.61195669 -1.1646947  -2.31956454
  0.5642974  -1.10766594  0.43282974 -0.20260256] and
 [ 0.81489992  0.76143959  0.07694935 -0.61195669 -1.1646947  -2.31956454
  0.5642974  -1.10766594  0.43282974 -0.20260256]
Covariance of samples [[ 0.95500506 -0.00345936]
 [-0.00345936  1.05820756]]


Let's look now at generating random numbers in parallel.

First, you need to start an ipcluster on your machine. This is done by running `ipcluster start` at the command line (in Windows PowerShell, for example). The code then sets up N random numbergenerators and stores them in `stream`, and also stores their initial state in `init_state`. First, the code generates N sets of M Gaussians in series, then reproduces one of the rows, and then generates N sets of M Gaussian in parallel.

In [12]:
def parseed():
    """
    A4.4 Page 166
    """
    N=4
    M=6
    sr=np.zeros((N,M))
    stream={};    init_state={}
    for j in range(N):
        stream[j]=np.random.RandomState() # create new rng
        init_state[j]=stream[j].get_state() # save initial state
        sr[j,:]=stream[j].randn(1,M)
    #
    def srandn(j): 
        stream[j].set_state(init_state[j])# reset state of jth rng
        return stream[j].randn(1,M)
    #
    sr4=srandn(N-1) # reproduces the 4th row        
    #
    from ipyparallel import Client
    import os
    rc = Client()   
    view = rc[:]
    # tell clients to work in our directory
    view.apply_sync(os.chdir, os.getcwd())
    ar=view.map_sync(lambda j: srandn(j),range(N))
    return sr,sr4,ar
#
sr,sr4,ar=parseed()
print("sr=",sr,"\nsr4=",sr4,"ar=\n",ar)

sr= [[ 0.90390365  0.65476072 -0.42647782  0.47217152  0.0518215  -0.32324357]
 [-0.24180847 -0.35042203 -1.21377586 -0.32739907 -1.74333538 -0.0759546 ]
 [ 1.27268467 -0.02500331 -0.0904326  -0.82035676  2.3166674   0.37689438]
 [ 0.05270123 -0.1900696  -0.66663564 -0.58521261 -0.03028957 -0.44856859]] 
sr4= [[ 0.05270123 -0.1900696  -0.66663564 -0.58521261 -0.03028957 -0.44856859]] ar=
 [array([[ 0.90390365,  0.65476072, -0.42647782,  0.47217152,  0.0518215 ,
        -0.32324357]]), array([[-0.24180847, -0.35042203, -1.21377586, -0.32739907, -1.74333538,
        -0.0759546 ]]), array([[ 1.27268467, -0.02500331, -0.0904326 , -0.82035676,  2.3166674 ,
         0.37689438]]), array([[ 0.05270123, -0.1900696 , -0.66663564, -0.58521261, -0.03028957,
        -0.44856859]])]


In [13]:
def uniform_ball():
    """
    A4.5 Page 167
    """
    theta=np.random.uniform()*2*pi; S=np.random.uniform()
    X=sqrt(S)*np.array([cos(theta),sin(theta)])
    return X

def uniform_sphere():
    """
    A4.6 Page 168
    """
    z=-1+2*np.random.uniform()
    theta=2*pi*np.random.uniform()
    r=sqrt(1-z*z)
    X=np.array([r*cos(theta),r*sin(theta),z])
    return X

def reject_uniform():
    """
    A4.7 Page 168
    """
    M=0; X=np.ones(2);
    while np.linalg.norm(X)>1:# reject
        M=M+1; X=np.random.uniform(-1,1,2) # generate sample
    return X,M

Here we have sampling methods for the uniform distributions on the ball and the sphere (by change of variables), with an alternative method for generating uniform samples on the unit ball (by rejection sampling)

In [14]:
print("Uniform sample from unit ball at origin", uniform_ball())
#
print("Uniform sample from unit sphere at origin", uniform_sphere())
#
[X,M]=reject_uniform()
print("Uniform sample of unit ball", X, "using", M, "attempts (rejection sampling)" )

Uniform sample from unit ball at origin [ 0.51655396  0.28705832]
Uniform sample from unit sphere at origin [ 0.5821194   0.25029924  0.7736196 ]
Uniform sample of unit ball [-0.10768545 -0.98238243] using 1 attempts (rejection sampling)


Sampling from multivariate Gaussians can be done by the Cholesky factorisation.

In [15]:
def gauss_chol(mu,C):
    """
    A4.8 Page 170
    """
    R= scipy.linalg.cholesky(C, lower=True)
    Z=np.random.randn(mu.size)
    X=mu+np.dot(R.T,Z)
    return X

Here's a $2\times 2$ example.

In [16]:
mu=np.array([0,0])
C=np.array([[2,1],[1,2]])
print("Multivariate Gaussian sample", gauss_chol(mu,C))

Multivariate Gaussian sample [-0.43260261  1.69354539]


We consider an ODE with a random initial data and compute the distribution of one of the components at time $T$, using a simple Monte Carlo method. See Example 4.72.

In [31]:
import ch3
def pop_monte(M,T,Dt,baru0,epsilon):
    """
    A4.9 Page 174
    """
    d=baru0.size
    N=int(T // Dt)
    u=np.zeros((M,d))  
    for j in range(M):
        u0=baru0 + epsilon * np.random.uniform(-1,1,baru0.size)
        t,usample=ch3.exp_euler(u0,T,N,d,f_pop)
        u[j,:]=usample[:,-1]
    bar_x,sig95=monte(u[:,0])
    return bar_x,sig95
    
def f_pop(u):
    return np.array([u[0]*(1-u[1]),
                     u[1]*(u[0]-1)])

def monte(samples):
    """
    Helper function for A4.9 Page 174
    """
    M=samples.size
    conf95=2 * sqrt(np.var(samples) / M)
    sample_av=np.mean(samples)
    return sample_av,conf95    

In [32]:
M=4*int(1e2); T=6
Dt=2e-3
baru0=np.array([0.5,2])
epsilon=0.2
bar_x,sig95=pop_monte(M,T,Dt,baru0,epsilon)
print("mean",bar_x,"sd",sig95)

mean 1.50227354061 sd 0.02939864970796154


By introducing a symmetry to the samples of the initial data, we reduce the variance in the quantity of interest.

In [33]:
def pop_monte_anti(M,T,Dt,baru0,epsilon):
    d=baru0.size
    N=int(T // Dt)
    u=np.zeros((2*M,d))  
    for j in range(M):
        u0=baru0 + epsilon * np.random.uniform(-1,1,baru0.size)
        t,usample=ch3.exp_euler(u0,T,N,d,f_pop)
        u[j,:]=usample[:,-1]
        u0=2 * baru0 - u0
        t,usample=ch3.exp_euler(u0,T,N,d,f_pop)
        u[j+M,:]=usample[:,-1]
    bar_x,sig95=monte(u[:,0])
    return bar_x,sig95

The resuling mean is similar, but the standard deviation is slightly reduced. Both runs are based on the same number of samples.

In [44]:
M=2*int(1e2); T=6
Dt=2e-3
baru0=np.array([0.5,2]); epsilon=0.2
bar_x,sig95=pop_monte_anti(M,T,Dt,baru0,epsilon)
print("mean",bar_x,"sd",sig95)

mean 1.49540169969 sd 0.026396530226
